In [ ]:
!pip install ultralytics lime
# !pip install ttach

In [ ]:
!git clone https://github.com/rigvedrs/YOLO-V11-CAM

In [ ]:
!cp -r /kaggle/working/YOLO-V11-CAM/yolo_cam  /kaggle/working

In [ ]:
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image
from ultralytics import YOLO

model = YOLO("/kaggle/input/detract-yolo11s/kaggle/working/runs/detect/train/weights/best.pt")

import cv2
import matplotlib.pyplot as plt
import numpy as np

img = cv2.imread('/kaggle/input/test-images/test_images/MVI_40863_img00081.jpg')  # або .png
#img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img)

annotated_img = results[0].plot()
#print(results)
plt.imshow(annotated_img)
plt.axis('off')
plt.show()

img = cv2.imread('/kaggle/input/test-images/test_images/MVI_40863_img00081.jpg')
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255

target_layers =[model.model.model[-2]]
cam = EigenCAM(model, target_layers,task='od')
grayscale_cam = cam(rgb_img)[0, :, :]
cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
plt.imshow(cam_image)
plt.show()

In [ ]:
!pip install lime

In [ ]:
import numpy as np
import cv2
from ultralytics import YOLO
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
from IPython.display import display, Image

def explain_yolo11_top3(image_path, model_path="/kaggle/input/detract-yolo11s/kaggle/working/runs/detect/train/weights/best.pt", num_samples=100, top_n=3):
    model = YOLO(model_path)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image_rgb)[0]
    detections = results.boxes.data.cpu().numpy()

    detections = detections[detections[:, 4].argsort()[::-1]]  # sort by confidence
    detections = detections[:top_n]

    explainer = lime_image.LimeImageExplainer()

    for idx, det in enumerate(detections):
        x1, y1, x2, y2, conf, cls = det.astype(int)
        label = model.names[int(cls)]

        crop_img = image_rgb[y1:y2, x1:x2]

        def yolo_predict(images):
            preds = []
            for img in images:
                img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                result = model(img_bgr, verbose=False)[0]
                boxes = result.boxes.data.cpu().numpy()
                cls_scores = np.zeros(len(model.names))
                for box in boxes:
                    if int(box[5]) == int(cls) and box[4] > 0.5:
                        cls_scores[int(cls)] = box[4]
                preds.append(cls_scores)
            return np.array(preds)

        explanation = explainer.explain_instance(
            image_rgb,
            yolo_predict,
            top_labels=1,
            hide_color=0,
            num_samples=num_samples
        )

        temp, mask = explanation.get_image_and_mask(
            explanation.top_labels[0],
            positive_only=True,
            num_features=5,
            hide_rest=False
        )

        # === Маленька фігура для crop-об’єкта ===
        plt.figure(figsize=(5, 5))
        plt.imshow(crop_img)
        plt.title(f'Detected Object: {label}', fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        plt.figure(figsize=(12, 12), dpi=150)
        plt.imshow(mark_boundaries(image_rgb, mask))
        plt.title(f'LIME Explanation: {label}', fontsize=18)
        plt.axis('off')

        plt.tight_layout()
        #plt.savefig(f'lime_expl_{idx}.png', dpi=300)
        plt.show()

if __name__ == "__main__":
    image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg" 
    explain_yolo11_top3(image_path, num_samples=100)

In [ ]:
image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg"
explain_yolo11_top3(image_path, num_samples=300)

In [ ]:
image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg"
explain_yolo11_top3(image_path, num_samples=500)

In [ ]:
image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg"
explain_yolo11_top3(image_path, num_samples=1000)

In [ ]:
image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg"
explain_yolo11_top3(image_path, num_samples=1500)

In [ ]:
image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg"
explain_yolo11_top3(image_path, num_samples=2000)

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shap
from ultralytics import YOLO
from PIL import Image


def explain_yolo11_detections(image_path, model, top_k=3, shap_samples=50):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)

    results = model(img_rgb, verbose=False)

    boxes = results[0].boxes.xyxy.cpu().numpy()  # Координати bounding boxes
    scores = results[0].boxes.conf.cpu().numpy()  # Впевненість
    classes = results[0].boxes.cls.cpu().numpy()  # Класи
    class_names = results[0].names  # Назви класів

    top_indices = np.argsort(scores)[::-1][:top_k]

    for idx in top_indices:
        class_id = int(classes[idx])
        class_name = class_names[class_id]
        score = scores[idx]
        box = boxes[idx]

        def model_predict(images, target_box=box, target_class_id=class_id):
            predictions = []
            for img in images:
                img = img.transpose((1, 2, 0))  
                img = img.astype(np.uint8)
                result = model(img, verbose=False)
                detected_boxes = result[0].boxes.xyxy.cpu().numpy()
                conf = result[0].boxes.conf.cpu().numpy()
                cls = result[0].boxes.cls.cpu().numpy()

                best_iou = 0
                best_conf = 0
                for i, (det_box, det_conf, det_cls) in enumerate(zip(detected_boxes, conf, cls)):
                    if int(det_cls) != target_class_id:
                        continue
                    x1 = max(target_box[0], det_box[0])
                    y1 = max(target_box[1], det_box[1])
                    x2 = min(target_box[2], det_box[2])
                    y2 = min(target_box[3], det_box[3])
                    intersection = max(0, x2 - x1) * max(0, y2 - y1)
                    area1 = (target_box[2] - target_box[0]) * (target_box[3] - target_box[1])
                    area2 = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
                    union = area1 + area2 - intersection
                    iou = intersection / union if union > 0 else 0
                    if iou > best_iou:
                        best_iou = iou
                        best_conf = det_conf
                predictions.append([best_conf]) 
            return np.array(predictions)

        masker = shap.maskers.Image("inpaint_telea", img_rgb.shape)

        explainer = shap.Explainer(model_predict, masker)

        img_shap = img_rgb.transpose((2, 0, 1))  
        img_shap = img_shap[np.newaxis, ...]  

        shap_values = explainer(img_shap, max_evals=shap_samples)

        print("SHAP values shape:", shap_values.shape)

        shap_vals = shap_values.values[0, ..., 0]  
        shap_vals = np.transpose(shap_vals, (1, 2, 0)) 
        print("Adjusted SHAP values shape:", shap_vals.shape)

        shap.image_plot(shap_vals, pixel_values=img_rgb, labels=[class_name], show=False)
        #plt.savefig(f'/kaggle/working/test{idx}_{shap_samples}.png', dpi=150)       
        
model = YOLO("/kaggle/input/detract-yolo11s/kaggle/working/runs/detect/train/weights/best.pt")

image_path = "/kaggle/input/test-images/test_images/MVI_40901_img00752.jpg" 
explain_yolo11_detections(image_path, model, top_k=3, shap_samples=50)

In [ ]:
explain_yolo11_detections(image_path, model, top_k=3, shap_samples=500)

In [ ]:
explain_yolo11_detections(image_path, model, top_k=3, shap_samples=1000)

In [ ]:
explain_yolo11_detections(image_path, model, top_k=3, shap_samples=150)

In [ ]:
explain_yolo11_detections(image_path, model, top_k=3, shap_samples=300)

In [ ]:
explain_yolo11_detections(image_path, model, top_k=3, shap_samples=500)